In [2]:
import pandas as pd
from numpy import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import gamma

from numpy import random as np_random

import hiplot as hip
from random import *
import random 
from numpy import random as np_random




# Plot settings
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['lines.linewidth'] = 2.0

# Set max row to 300
pd.set_option('display.max_rows', 300)

In [ ]:
# define simulation function
def simulate_path(s0, mu, sigma, horizon=1, timesteps=252, n_sims=10000,number_of_rounds=2,period_rounds=[0,252-1],rounds=[0,0]):

    # set the random seed for reproducibility
    random.seed(42)

    # read parameters
    S0 = s0                                   # initial spot price
    r = mu                                    # mu = rf in risk neutral framework
    T = horizon                               # time horizon
    t = timesteps                             # number of time steps
    n = n_sims                                # number of simulation
    n_rounds = number_of_rounds               # number of simulation
    p_round = period_rounds                   # period of each round
    rounds= rounds                            # amount of each round

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    S = zeros((t,n))
    S[0] = S0
    

    #initial step to Round A -1 step  
    for i in range(0,p_round[0] -1):
        w= np.random.normal(0.2, 6.0, n)
        S[i+1] = S[i] * (1 + r*dt + sigma*sqrt(dt)*w)
        

    
    
    #Round A
    for i in range(0,n):
        S[p_round[0]][i] = S[p_round[0]-1][i] + rounds[0]


    #Round A + 1 step to Round B -1 step         
    for i in range(p_round[0]+1, p_round[1]):
        w= np.random.normal(0.2, 6.0, n)
        S[i] = S[i-1] * (1 + r*dt + sigma*sqrt(dt)*w)        


    #Round B
    for i in range(0,n):
        S[p_round[1]][i] = S[p_round[1]-1][i] + rounds[1]


    #Round B + 1 step to final step 
    for i in range(p_round[1] +1,t):   
        w= np.random.normal(0.2, 6.0, n)
        S[i] = S[i-1] * (1 + r*dt + sigma*sqrt(dt)*w)

    S[S<0] = 0 

    return S
        


################

# ajustar distribuição para simulação. 
# colocar mais rounds
# colocar downrounds - OK
# fazer simulação com inputs consorciei 

In [ ]:
# Random numbers drawn from a standard normal distribution
random.seed(42)
w_sn = np.random.standard_normal(10^4)
w_sn.shape, w_sn.mean(), w_sn.std()


#Parameters
freq_rounds = [24,60]
exit_period= 96
premoney_period= [n-1 for n in freq_rounds]
fund_rounds = [5*10**6, 20*10**6]


#Data gen. 
price_path = pd.DataFrame(simulate_path(10**6,0.1308,0.20,15,12*15,10**5,2,freq_rounds,fund_rounds))

In [ ]:
dataset= pd.DataFrame()

#valuations pre money
dataset["pre_A"]=pd.DataFrame(price_path.loc[premoney_period[0]:premoney_period[0],:].values.T)
dataset["pre_B"]=pd.DataFrame(price_path.loc[premoney_period[1]:premoney_period[1],:].values.T)

#valuations post money
dataset["post_A"]=pd.DataFrame(price_path.loc[freq_rounds[0]:freq_rounds[0],:].values.T)
dataset["post_B"]=pd.DataFrame(price_path.loc[freq_rounds[1]:freq_rounds[1],:].values.T)

#cap table 
dataset["cap_A"]=   fund_rounds[0] / dataset["post_A"]   
dataset["cap_B"]=   fund_rounds[1] / dataset["post_B"] 
dataset["cap_A_final"]= dataset["cap_A"]*dataset["pre_B"]/dataset["post_B"]

#liquidation preference and % participation     
dataset["lp_A"]= np.random.randint(1, 4, dataset.shape[0])

dataset["part_A"]=np.random.uniform(0, 1, dataset.shape[0])

dataset["lp_B"]=np.random.randint(1, 4, dataset.shape[0])

dataset["part_B"]=np.random.uniform(0, 1, dataset.shape[0])

#exit valuation
dataset["exit"]=pd.DataFrame(price_path.loc[exit_period:exit_period,:].values.T)

#returns and moics
dataset["ret_B"]= np.where(dataset["exit"]>= fund_rounds[1]*dataset["lp_B"],
                           (pd.DataFrame({'n1': (dataset["exit"]*dataset["cap_B"]).values,'n2': (fund_rounds[1]*dataset["lp_B"]).values})).max(axis=1),
                           (pd.DataFrame({'n1': (dataset["lp_B"]*fund_rounds[1]).values,'n2': (dataset["exit"]).values})).max(axis=1))

dataset["moic_B"]= dataset["ret_B"]/fund_rounds[1]

dataset["ret_A"]= np.where((dataset["exit"]-dataset["ret_B"])>= fund_rounds[0]*dataset["lp_A"],
                           (pd.DataFrame({'n1': ((dataset["exit"]-dataset["ret_B"])*dataset["cap_A_final"]).values,'n2': (fund_rounds[0]*dataset["lp_A"]).values})).max(axis=1),
                           (pd.DataFrame({'n1': (dataset["lp_A"]*fund_rounds[0]).values,'n2': ((dataset["exit"]-dataset["ret_B"])).values})).max(axis=1))

dataset["moic_A"]= dataset["ret_A"]/fund_rounds[0]


#liquidation preference value
dataset["liq_p_B"]= np.where((dataset["ret_B"]-(dataset["cap_B"]*dataset["exit"]))>= fund_rounds[1]*dataset["lp_B"],
                           0,
                           dataset["ret_B"]-(dataset["cap_B"]*dataset["exit"]))


dataset["liq_p_A"]= np.where((dataset["ret_A"]-(dataset["cap_A_final"]*(dataset["exit"]-dataset["ret_B"])))>= fund_rounds[0]*dataset["lp_A"],
                           0,
                           dataset["ret_A"]-(dataset["cap_A_final"]*(dataset["exit"]-dataset["ret_B"])))






#dataset["valor_lp"]=[]

#dataset["moic"]=[]



#c=(pd.DataFrame({'n1': (dataset["exit"]*dataset["cap_B"]).values,
#                'n2': (fund_rounds[1]*dataset["lp_B"]).values})).max(axis=1)


In [ ]:
df = ((dataset[(dataset.cap_B>0.01) & (dataset.cap_B<=1) & (dataset.cap_A>0.01) & (dataset.cap_A<=1) & (dataset.cap_A_final + dataset.cap_B <1)] ).sort_values("moic_B",ascending=True))

In [ ]:
data = df.to_dict(orient='records')

hip.Experiment.from_iterable(data).display()